In [1]:
import cv2
import logging
import os
import random
import time
import warnings

import librosa
import librosa.display as display
import numpy as np
import pandas as pd

from pathlib import Path

from collections import defaultdict
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import label_ranking_average_precision_score

import tensorflow as tf
from tensorflow import keras


In [2]:
kaggle_birdsong_meta = pd.read_csv('/media/wwymak/Storage2/birdsong_dataset/kaggle_birdsong_recognition/train.csv')
kaggle_birdsong_meta.head()

,rating,playback_used,ebird_code,channels,date,pitch,duration,filename,speed,species,...,xc_id,url,country,author,primary_label,longitude,length,time,recordist,license
0,3.5,no,aldfly,1 (mono),2013-05-25,Not specified,25,XC134874.mp3,Not specified,Alder Flycatcher,...,134874,https://www.xeno-canto.org/134874,United States,Jonathon Jongsma,Empidonax alnorum_Alder Flycatcher,-92.962,Not specified,8:00,Jonathon Jongsma,Creative Commons Attribution-ShareAlike 3.0
1,4.0,no,aldfly,2 (stereo),2013-05-27,both,36,XC135454.mp3,both,Alder Flycatcher,...,135454,https://www.xeno-canto.org/135454,United States,Mike Nelson,Empidonax alnorum_Alder Flycatcher,-82.1106,0-3(s),08:30,Mike Nelson,Creative Commons Attribution-NonCommercial-Sha...
2,4.0,no,aldfly,2 (stereo),2013-05-27,both,39,XC135455.mp3,both,Alder Flycatcher,...,135455,https://www.xeno-canto.org/135455,United States,Mike Nelson,Empidonax alnorum_Alder Flycatcher,-82.1106,0-3(s),08:30,Mike Nelson,Creative Commons Attribution-NonCommercial-Sha...
3,3.5,no,aldfly,2 (stereo),2013-05-27,both,33,XC135456.mp3,both,Alder Flycatcher,...,135456,https://www.xeno-canto.org/135456,United States,Mike Nelson,Empidonax alnorum_Alder Flycatcher,-82.1106,0-3(s),08:30,Mike Nelson,Creative Commons Attribution-NonCommercial-Sha...
4,4.0,no,aldfly,2 (stereo),2013-05-27,both,36,XC135457.mp3,level,Alder Flycatcher,...,135457,https://www.xeno-canto.org/135457,United States,Mike Nelson,Empidonax alnorum_Alder Flycatcher,-82.1106,0-3(s),08:30,Mike Nelson,Creative Commons Attribution-NonCommercial-Sha...


In [5]:
kaggle_birdsong_meta.columns

Index(['rating', 'playback_used', 'ebird_code', 'channels', 'date', 'pitch',
       'duration', 'filename', 'speed', 'species', 'number_of_notes', 'title',
       'secondary_labels', 'bird_seen', 'sci_name', 'location', 'latitude',
       'sampling_rate', 'type', 'elevation', 'description', 'bitrate_of_mp3',
       'file_type', 'volume', 'background', 'xc_id', 'url', 'country',
       'author', 'primary_label', 'longitude', 'length', 'time', 'recordist',
       'license'],
      dtype='object')

In [8]:
kaggle_birdsong_meta.isnull().sum()/len(kaggle_birdsong_meta)

rating              0.000000
playback_used       0.084211
ebird_code          0.000000
channels            0.000000
date                0.000000
pitch               0.000000
duration            0.000000
filename            0.000000
speed               0.000000
species             0.000000
number_of_notes     0.000000
title               0.000000
secondary_labels    0.000000
bird_seen           0.084211
sci_name            0.000000
location            0.000000
latitude            0.000000
sampling_rate       0.000000
type                0.000000
elevation           0.000000
description         0.290012
bitrate_of_mp3      0.000374
file_type           0.000000
volume              0.000000
background          0.611696
xc_id               0.000000
url                 0.000000
country             0.000000
author              0.000000
primary_label       0.000000
longitude           0.000000
length              0.000000
time                0.000000
recordist           0.000000
license       

In [5]:
kaggle_birdsong_meta.country.value_counts().head(30)

United States         14284
Canada                 2516
Mexico                 1099
Sweden                  269
Ecuador                 234
Russian Federation      210
France                  204
Poland                  177
Honduras                166
Brazil                  163
Colombia                149
Norway                  142
Finland                 137
United Kingdom          120
Argentina               118
Germany                 115
Netherlands             113
Iceland                 103
Japan                    96
Spain                    69
Costa Rica               64
El Salvador              63
Switzerland              58
Nicaragua                38
Portugal                 37
Italy                    37
Kazakhstan               34
Panama                   34
Guatemala                33
Belgium                  28
Name: country, dtype: int64

In [11]:
uk_birds = kaggle_birdsong_meta[kaggle_birdsong_meta.country=='United Kingdom'].ebird_code.unique()
other_birds = kaggle_birdsong_meta[kaggle_birdsong_meta.country!='United Kingdom'].ebird_code.unique()


In [10]:
len(uk_birds), len(set(uk_birds).intersection(set(other_birds))), len(other_birds)

(30, 30, 264)

In [12]:
ff1010_bird_metadata = pd.read_csv('/media/wwymak/Storage2/birdsong_dataset/ff1010bird_metadata.csv')
ff1010_bird_metadata.head()

,itemid,hasbird
0,64486,0
1,2525,0
2,44981,0
3,101323,0
4,165746,0


In [13]:
ff1010_bird_metadata.hasbird.value_counts()

0    5755
1    1935
Name: hasbird, dtype: int64